### Import 
- libraries

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import math

- main dataset
- coordinates for hdb
- coordinates for shopping mall
- coordinates for mrt/lrt

In [3]:
df = pd.read_csv('c:/Users/Admin/Desktop/individual_project/address_added.csv') #main dataset
mrt_df = pd.read_csv('c:/Users/Admin/Desktop/individual_project/mrt_lrt_data.csv') #mrt/lrt dataset
hdb_df = pd.read_csv('c:/Users/Admin/Desktop/individual_project/hdb_coordinates.csv') #hdb coordinates dataset
shopping_mall_df = pd.read_csv('c:/Users/Admin/Desktop/individual_project/shopping_mall_coordinates.csv') #shopping mall coordinates

In [4]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address
0,2017-01-01,ang mo kio,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979-01-01,61 years 04 months,232000.0,406 ANG MO KIO AVE 10
1,2017-01-01,ang mo kio,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978-01-01,60 years 07 months,250000.0,108 ANG MO KIO AVE 4
2,2017-01-01,ang mo kio,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980-01-01,62 years 05 months,262000.0,602 ANG MO KIO AVE 5
3,2017-01-01,ang mo kio,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980-01-01,62 years 01 month,265000.0,465 ANG MO KIO AVE 10
4,2017-01-01,ang mo kio,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980-01-01,62 years 05 months,265000.0,601 ANG MO KIO AVE 5


In [5]:
mrt_df.head()

,station_name,type,lat,lng
0,Jurong East,MRT,1.333207,103.742308
1,Bukit Batok,MRT,1.349069,103.749596
2,Bukit Gombak,MRT,1.359043,103.751863
3,Choa Chu Kang,MRT,1.385417,103.744316
4,Yew Tee,MRT,1.397383,103.747523


In [6]:
hdb_df.head()

,latitude,longitude,blk_no,road_name,postal_code,address
0,1.362005,103.853880,406,ANG MO KIO AVENUE 10,560406,406 ANG MO KIO AVE 10
1,1.370966,103.838202,108,ANG MO KIO AVENUE 4,560108,108 ANG MO KIO AVE 4
2,1.380709,103.835368,602,ANG MO KIO AVENUE 5,560602,602 ANG MO KIO AVE 5
3,1.366201,103.857201,465,ANG MO KIO AVENUE 10,560465,465 ANG MO KIO AVE 10
4,1.381041,103.835132,601,ANG MO KIO AVENUE 5,560601,601 ANG MO KIO AVE 5


In [7]:
shopping_mall_df.head()

,Mall Name,LATITUDE,LONGITUDE
0,100 AM,1.274588,103.843471
1,313@Somerset,1.301014,103.838361
2,Aperia,1.310474,103.864313
3,Balestier Hill Shopping Centre,1.325596,103.842572
4,Bugis Cube,1.298141,103.855635


## Changing mrt_df and shopping_mall_df columns name to be the same as hdb_df

In [8]:
#shopping mall
shopping_mall_df.columns = ['mall_name', 'latitude', 'longitude']

#mrt
mrt_df.columns = ['station_name', 'type', 'latitude', 'longitude']


In [9]:
shopping_mall_df.head()

,mall_name,latitude,longitude
0,100 AM,1.274588,103.843471
1,313@Somerset,1.301014,103.838361
2,Aperia,1.310474,103.864313
3,Balestier Hill Shopping Centre,1.325596,103.842572
4,Bugis Cube,1.298141,103.855635


In [10]:
mrt_df.head()

,station_name,type,latitude,longitude
0,Jurong East,MRT,1.333207,103.742308
1,Bukit Batok,MRT,1.349069,103.749596
2,Bukit Gombak,MRT,1.359043,103.751863
3,Choa Chu Kang,MRT,1.385417,103.744316
4,Yew Tee,MRT,1.397383,103.747523


## Finding the nearest mrt and shopping mall for each HDB values.

-result_df is the final outcome


In [11]:
# Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of Earth in kilometers
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Add nearest MRT station and mall to HDB DataFrame
def find_nearest_locations(hdb_df, mrt_df, shopping_mall_df):
    nearest_mrt = []
    nearest_mall = []
    nearest_mrt_distance = []
    nearest_mall_distance = []

    for _, hdb_row in hdb_df.iterrows():
        hdb_lat, hdb_lon = hdb_row["latitude"], hdb_row["longitude"]

        # Calculate distance to all MRT stations and find the nearest
        mrt_df["distance"] = mrt_df.apply(lambda x: haversine(hdb_lat, hdb_lon, x["latitude"], x["longitude"]), axis=1)
        nearest_mrt_row = mrt_df.loc[mrt_df["distance"].idxmin()]
        nearest_mrt.append(nearest_mrt_row["station_name"])
        nearest_mrt_distance.append(nearest_mrt_row["distance"])

        # Calculate distance to all shopping malls and find the nearest
        shopping_mall_df["distance"] = shopping_mall_df.apply(lambda x: haversine(hdb_lat, hdb_lon, x["latitude"], x["longitude"]), axis=1)
        nearest_mall_row = shopping_mall_df.loc[shopping_mall_df["distance"].idxmin()]
        nearest_mall.append(nearest_mall_row["mall_name"])
        nearest_mall_distance.append(nearest_mall_row["distance"])

    # Add the results to the HDB DataFrame
    hdb_df["nearest_mrt_station"] = nearest_mrt
    hdb_df["nearest_mrt_distance_km"] = nearest_mrt_distance
    hdb_df["nearest_mall"] = nearest_mall
    hdb_df["nearest_mall_distance_km"] = nearest_mall_distance

    return hdb_df

result_df = find_nearest_locations(hdb_df, mrt_df, shopping_mall_df)
result_df.head()

,latitude,longitude,blk_no,road_name,postal_code,address,nearest_mrt_station,nearest_mrt_distance_km,nearest_mall,nearest_mall_distance_km
0,1.362005,103.853880,406,ANG MO KIO AVENUE 10,560406,406 ANG MO KIO AVE 10,Ang Mo Kio,1.011433,AMK Hub,1.006211
1,1.370966,103.838202,108,ANG MO KIO AVENUE 4,560108,108 ANG MO KIO AVE 4,Ang Mo Kio,1.270031,Broadway Plaza,0.867977
2,1.380709,103.835368,602,ANG MO KIO AVENUE 5,560602,602 ANG MO KIO AVE 5,Yio Chu Kang,1.068607,Broadway Plaza,1.528020
3,1.366201,103.857201,465,ANG MO KIO AVENUE 10,560465,465 ANG MO KIO AVE 10,Ang Mo Kio,0.947097,myVillage At Serangoon Garden,0.892903
4,1.381041,103.835132,601,ANG MO KIO AVENUE 5,560601,601 ANG MO KIO AVE 5,Yio Chu Kang,1.091392,Broadway Plaza,1.571900


- download result_df into a csv

In [ ]:
#result_df.to_csv('c:/Users/Admin/Desktop/individual_project/result_df.csv', index = False)

## Left joining result_df to df
- changing df column names to be the same as result_df

In [16]:
df.columns = ['month', 'town', 'flat_type', 'block_no', 'road_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease','resale_price', 'address' ]

In [17]:
df.head()

,month,town,flat_type,block_no,road_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address
0,2017-01-01,ang mo kio,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979-01-01,61 years 04 months,232000.0,406 ANG MO KIO AVE 10
1,2017-01-01,ang mo kio,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978-01-01,60 years 07 months,250000.0,108 ANG MO KIO AVE 4
2,2017-01-01,ang mo kio,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980-01-01,62 years 05 months,262000.0,602 ANG MO KIO AVE 5
3,2017-01-01,ang mo kio,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980-01-01,62 years 01 month,265000.0,465 ANG MO KIO AVE 10
4,2017-01-01,ang mo kio,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980-01-01,62 years 05 months,265000.0,601 ANG MO KIO AVE 5


- left join result_df to df

In [20]:
# Perform the left join
merged_df = pd.merge(
    df,
    result_df,
    how="left",  # Left join to keep all rows in df
    left_on=["address"],  # Keys in df
    right_on=["address"]  # Keys in result_df
)

# # Drop duplicate columns if necessary
# merged_df = merged_df.drop(columns=["blk_no", "road_name"])

# Display the merged DataFrame
merged_df.head()

,month,town,flat_type,block_no,road_name_x,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,address,latitude,longitude,blk_no,road_name_y,postal_code,nearest_mrt_station,nearest_mrt_distance_km,nearest_mall,nearest_mall_distance_km
0,2017-01-01,ang mo kio,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979-01-01,61 years 04 months,...,406 ANG MO KIO AVE 10,1.362005,103.853880,406,ANG MO KIO AVENUE 10,560406,Ang Mo Kio,1.011433,AMK Hub,1.006211
1,2017-01-01,ang mo kio,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978-01-01,60 years 07 months,...,108 ANG MO KIO AVE 4,1.370966,103.838202,108,ANG MO KIO AVENUE 4,560108,Ang Mo Kio,1.270031,Broadway Plaza,0.867977
2,2017-01-01,ang mo kio,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980-01-01,62 years 05 months,...,602 ANG MO KIO AVE 5,1.380709,103.835368,602,ANG MO KIO AVENUE 5,560602,Yio Chu Kang,1.068607,Broadway Plaza,1.528020
3,2017-01-01,ang mo kio,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980-01-01,62 years 01 month,...,465 ANG MO KIO AVE 10,1.366201,103.857201,465,ANG MO KIO AVENUE 10,560465,Ang Mo Kio,0.947097,myVillage At Serangoon Garden,0.892903
4,2017-01-01,ang mo kio,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980-01-01,62 years 05 months,...,601 ANG MO KIO AVE 5,1.381041,103.835132,601,ANG MO KIO AVENUE 5,560601,Yio Chu Kang,1.091392,Broadway Plaza,1.571900


## Clean merged_df
- remove duplicate column and rename road_name_y back to streetname


In [23]:
merged_df = merged_df.drop(columns=["road_name_x", 'blk_no'])
merged_df.columns = merged_df.columns.str.replace("road_name_y", "street_name")

In [24]:
merged_df.head()

,month,town,flat_type,block_no,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,latitude,longitude,street_name,postal_code,nearest_mrt_station,nearest_mrt_distance_km,nearest_mall,nearest_mall_distance_km
0,2017-01-01,ang mo kio,2 ROOM,406,10 TO 12,44.0,Improved,1979-01-01,61 years 04 months,232000.0,406 ANG MO KIO AVE 10,1.362005,103.853880,ANG MO KIO AVENUE 10,560406,Ang Mo Kio,1.011433,AMK Hub,1.006211
1,2017-01-01,ang mo kio,3 ROOM,108,01 TO 03,67.0,New Generation,1978-01-01,60 years 07 months,250000.0,108 ANG MO KIO AVE 4,1.370966,103.838202,ANG MO KIO AVENUE 4,560108,Ang Mo Kio,1.270031,Broadway Plaza,0.867977
2,2017-01-01,ang mo kio,3 ROOM,602,01 TO 03,67.0,New Generation,1980-01-01,62 years 05 months,262000.0,602 ANG MO KIO AVE 5,1.380709,103.835368,ANG MO KIO AVENUE 5,560602,Yio Chu Kang,1.068607,Broadway Plaza,1.528020
3,2017-01-01,ang mo kio,3 ROOM,465,04 TO 06,68.0,New Generation,1980-01-01,62 years 01 month,265000.0,465 ANG MO KIO AVE 10,1.366201,103.857201,ANG MO KIO AVENUE 10,560465,Ang Mo Kio,0.947097,myVillage At Serangoon Garden,0.892903
4,2017-01-01,ang mo kio,3 ROOM,601,01 TO 03,67.0,New Generation,1980-01-01,62 years 05 months,265000.0,601 ANG MO KIO AVE 5,1.381041,103.835132,ANG MO KIO AVENUE 5,560601,Yio Chu Kang,1.091392,Broadway Plaza,1.571900


In [25]:
merged_df.isnull().sum()

month                       0
town                        0
flat_type                   0
block_no                    0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
remaining_lease             0
resale_price                0
address                     0
latitude                    0
longitude                   0
street_name                 0
postal_code                 0
nearest_mrt_station         0
nearest_mrt_distance_km     0
nearest_mall                0
nearest_mall_distance_km    0
dtype: int64

In [29]:
duplicated_rows = merged_df.duplicated()
print(duplicated_rows.sum())


0


## Changing values in storey_range

In [27]:
merged_df['storey_range'].unique()


array(['10 TO 12', '01 TO 03', '04 TO 06', '07 TO 09', '13 TO 15',
       '19 TO 21', '22 TO 24', '16 TO 18', '34 TO 36', '28 TO 30',
       '37 TO 39', '49 TO 51', '25 TO 27', '40 TO 42', '31 TO 33',
       '46 TO 48', '43 TO 45'], dtype=object)

In [30]:
# Create a mapping dictionary
mapping = {
    "10 TO 12": "11",
    "01 TO 03": "2",
    "04 TO 06": "5",
    "07 TO 09": "8",
    "13 TO 15": "14",
    "19 TO 21": "20",
    "22 TO 24": "23",
    "16 TO 18": "17",
    "34 TO 36": "35",
    "28 TO 30": "29",
    "37 TO 39": "38",
    "49 TO 51": "50",
    "25 TO 27": "26",
    "40 TO 42": "41",
    "31 TO 33": "32",
    "46 TO 48": "47",
    "43 TO 45": "44"
}

# Replace the values in the column using the mapping
merged_df["storey_range"] = merged_df["storey_range"].replace(mapping)

# Display the updated DataFrame
merged_df.head()


,month,town,flat_type,block_no,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,latitude,longitude,street_name,postal_code,nearest_mrt_station,nearest_mrt_distance_km,nearest_mall,nearest_mall_distance_km
0,2017-01-01,ang mo kio,2 ROOM,406,11,44.0,Improved,1979-01-01,61 years 04 months,232000.0,406 ANG MO KIO AVE 10,1.362005,103.853880,ANG MO KIO AVENUE 10,560406,Ang Mo Kio,1.011433,AMK Hub,1.006211
1,2017-01-01,ang mo kio,3 ROOM,108,2,67.0,New Generation,1978-01-01,60 years 07 months,250000.0,108 ANG MO KIO AVE 4,1.370966,103.838202,ANG MO KIO AVENUE 4,560108,Ang Mo Kio,1.270031,Broadway Plaza,0.867977
2,2017-01-01,ang mo kio,3 ROOM,602,2,67.0,New Generation,1980-01-01,62 years 05 months,262000.0,602 ANG MO KIO AVE 5,1.380709,103.835368,ANG MO KIO AVENUE 5,560602,Yio Chu Kang,1.068607,Broadway Plaza,1.528020
3,2017-01-01,ang mo kio,3 ROOM,465,5,68.0,New Generation,1980-01-01,62 years 01 month,265000.0,465 ANG MO KIO AVE 10,1.366201,103.857201,ANG MO KIO AVENUE 10,560465,Ang Mo Kio,0.947097,myVillage At Serangoon Garden,0.892903
4,2017-01-01,ang mo kio,3 ROOM,601,2,67.0,New Generation,1980-01-01,62 years 05 months,265000.0,601 ANG MO KIO AVE 5,1.381041,103.835132,ANG MO KIO AVENUE 5,560601,Yio Chu Kang,1.091392,Broadway Plaza,1.571900


In [ ]:
# merged_df.to_csv('c:/Users/Admin/Desktop/individual_project/merged_df.csv', index = False)

## Finding distance between HDB and CBD

In [4]:
merged_df = pd.read_csv('c:/Users/Admin/Desktop/individual_project/merged_df.csv')

C:\Users\Admin\AppData\Local\Temp\ipykernel_10216\787237713.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('c:/Users/Admin/Desktop/individual_project/merged_df.csv')


In [35]:
merged_df.head()

,month,town,flat_type,block_no,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,latitude,longitude,street_name,postal_code,nearest_mrt_station,nearest_mrt_distance_km,nearest_mall,nearest_mall_distance_km
0,2017-01-01,ang mo kio,2 ROOM,406,11,44.0,Improved,1979-01-01,61 years 04 months,232000.0,406 ANG MO KIO AVE 10,1.362005,103.853880,ANG MO KIO AVENUE 10,560406,Ang Mo Kio,1.011433,AMK Hub,1.006211
1,2017-01-01,ang mo kio,3 ROOM,108,2,67.0,New Generation,1978-01-01,60 years 07 months,250000.0,108 ANG MO KIO AVE 4,1.370966,103.838202,ANG MO KIO AVENUE 4,560108,Ang Mo Kio,1.270031,Broadway Plaza,0.867977
2,2017-01-01,ang mo kio,3 ROOM,602,2,67.0,New Generation,1980-01-01,62 years 05 months,262000.0,602 ANG MO KIO AVE 5,1.380709,103.835368,ANG MO KIO AVENUE 5,560602,Yio Chu Kang,1.068607,Broadway Plaza,1.528020
3,2017-01-01,ang mo kio,3 ROOM,465,5,68.0,New Generation,1980-01-01,62 years 01 month,265000.0,465 ANG MO KIO AVE 10,1.366201,103.857201,ANG MO KIO AVENUE 10,560465,Ang Mo Kio,0.947097,myVillage At Serangoon Garden,0.892903
4,2017-01-01,ang mo kio,3 ROOM,601,2,67.0,New Generation,1980-01-01,62 years 05 months,265000.0,601 ANG MO KIO AVE 5,1.381041,103.835132,ANG MO KIO AVENUE 5,560601,Yio Chu Kang,1.091392,Broadway Plaza,1.571900


In [36]:
# Define CBD coordinates 
cbd_latitude = 1.287953
cbd_longitude = 103.851784

merged_df["min_dist_to_cbd"] = merged_df.apply(
    lambda row: haversine(row["latitude"], row["longitude"], cbd_latitude, cbd_longitude), axis=1
)

merged_df.tail()


,month,town,flat_type,block_no,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,latitude,longitude,street_name,postal_code,nearest_mrt_station,nearest_mrt_distance_km,nearest_mall,nearest_mall_distance_km,min_dist_to_cbd
193871,2024-11-01,yishun,5 ROOM,235,8,135.0,Model A,1985-01-01,59 years 10 months,686000.0,235 YISHUN ST 21,1.434229,103.838544,YISHUN STREET 21,760235,Yishun,0.639369,Junction Nine,0.367665,16.331592
193872,2024-11-01,yishun,5 ROOM,335C,8,112.0,Improved,2015-01-01,90 years 01 month,700000.0,335C YISHUN ST 31,1.432488,103.844367,YISHUN STREET 31,763335,Yishun,1.082843,Junction Nine,0.309019,16.092666
193873,2024-11-01,yishun,5 ROOM,511B,11,113.0,Improved,2017-01-01,91 years 10 months,720000.0,511B YISHUN ST 51,1.415600,103.842414,YISHUN STREET 51,762511,Khatib,1.066478,Wisteria Mall,0.292344,14.231882
193874,2024-11-01,yishun,EXECUTIVE,405,5,148.0,Maisonette,1988-01-01,62 years 10 months,855500.0,405 YISHUN AVE 6,1.426840,103.849274,YISHUN AVENUE 6,760405,Yishun,1.612668,Junction Nine,1.097326,15.446065
193875,2024-11-01,yishun,EXECUTIVE,877,2,142.0,Apartment,1987-01-01,62 years 03 months,865000.0,877 YISHUN ST 81,1.413902,103.835454,YISHUN STREET 81,760877,Khatib,0.477475,Wisteria Mall,0.794602,14.122072


In [5]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193876 entries, 0 to 193875
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   month                     193876 non-null  object 
 1   town                      193876 non-null  object 
 2   flat_type                 193876 non-null  object 
 3   block_no                  193876 non-null  object 
 4   storey_range              193876 non-null  int64  
 5   floor_area_sqm            193876 non-null  float64
 6   flat_model                193876 non-null  object 
 7   lease_commence_date       193876 non-null  object 
 8   remaining_lease           193876 non-null  object 
 9   resale_price              193876 non-null  float64
 10  address                   193876 non-null  object 
 11  latitude                  193876 non-null  float64
 12  longitude                 193876 non-null  float64
 13  street_name               193876 non-null  o

## Extracting remaining years of lease

In [ ]:
# extract the first integer out(which is the year)
merged_df['remaining_year'] = merged_df['remaining_lease'].str.extract(r'(\d+)').astype(int)


In [7]:
merged_df.tail()

,month,town,flat_type,block_no,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,...,latitude,longitude,street_name,postal_code,nearest_mrt_station,nearest_mrt_distance_km,nearest_mall,nearest_mall_distance_km,min_dist_to_cbd,remaining_year
193871,2024-11-01,yishun,5 ROOM,235,8,135.0,Model A,1985-01-01,59 years 10 months,686000.0,...,1.434229,103.838544,YISHUN STREET 21,760235,Yishun,0.639369,Junction Nine,0.367665,16.331592,59
193872,2024-11-01,yishun,5 ROOM,335C,8,112.0,Improved,2015-01-01,90 years 01 month,700000.0,...,1.432488,103.844367,YISHUN STREET 31,763335,Yishun,1.082843,Junction Nine,0.309019,16.092666,90
193873,2024-11-01,yishun,5 ROOM,511B,11,113.0,Improved,2017-01-01,91 years 10 months,720000.0,...,1.415600,103.842414,YISHUN STREET 51,762511,Khatib,1.066478,Wisteria Mall,0.292344,14.231882,91
193874,2024-11-01,yishun,EXECUTIVE,405,5,148.0,Maisonette,1988-01-01,62 years 10 months,855500.0,...,1.426840,103.849274,YISHUN AVENUE 6,760405,Yishun,1.612668,Junction Nine,1.097326,15.446065,62
193875,2024-11-01,yishun,EXECUTIVE,877,2,142.0,Apartment,1987-01-01,62 years 03 months,865000.0,...,1.413902,103.835454,YISHUN STREET 81,760877,Khatib,0.477475,Wisteria Mall,0.794602,14.122072,62
